<a href="https://colab.research.google.com/github/rm3887/Design-Project-/blob/master/NLU_pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **MAKE SURE that you change runtime type from None to GPU under Runtime above**

In [9]:
import io
import pandas as pd
import numpy as np

df = pd.read_csv(io.BytesIO(uploaded['comments.csv']), header = None, names = ['Text'])
df.head(5)

,Text
0,https://www.latimes.com/california/story/2020-...
1,https://twitter.com/LASDHQ/status/122151440905...
2,"I can’t believe it, the guy has flown in helic..."
3,https://www.tmz.com/2020/01/26/kobe-bryant-kil...
4,Holy shit. This can’t be true


# Don't forget to change import tensorflow as tf to:
#import tensorflow.compat.v1 as tf
#tf.compat.v1.disable_v2_behavior()

We do this for optimization.py, run_classifier.py, create_pretraining_data.py, tokentization.py

If any errors, do this first

In [0]:
import os
import sys
import shutil
import argparse
import tempfile
import urllib.request
import zipfile
import json

In [6]:
!test -d bert_repo || git clone https://github.com/rm3887/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

import tensorflow.compat.v1 as tf

import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

Instructions for updating:
non-resource variables are not supported in the long term


In [0]:
url = 'https://raw.githubusercontent.com/rm3887/NLU-Project-PreTraining-RoBERTa/master/comments.csv'

In [0]:
import io
import pandas as pd
import numpy as np

have to add the additional tokens to pretrain the model later on

In [0]:
df = pd.read_csv(url, error_bad_lines=False)
additionals = ['[PAD]', '[EOS]' ,'[UNK]', '[CLS]',  '[SEP]', '[MASK]']
for word in additionals:
  df = df.append({'Text': word},ignore_index=True)

In [10]:
df

,Text
0,https://www.latimes.com/california/story/2020-...
1,https://twitter.com/LASDHQ/status/122151440905...
2,https://twitter.com/abc7curt/status/1221517701...
3,TMZ are known to be reliable when it comes to ...
4,Already up on their IG. TMZ does not fail in s...
...,...
299306,[EOS]
299307,[UNK]
299308,[CLS]
299309,[SEP]


## Create our own word embeddings from the pretraining dataset so that our BERT model will pick up on common phrases used.

In [0]:
from gensim.models import Word2Vec
import re

In [0]:
sentences = [re.findall(r"[\S']+", df.iloc[i,0]) for i in range(len(df))]

In [0]:
model = Word2Vec(sentences, min_count=1)

In [0]:
vocab = list(model.wv.vocab)

In [0]:
df.to_csv('/content/training_data.tsv', sep='\t', index=False, header=False)

In [0]:
with open('/content/vocab.txt', 'w+') as f:
  for word in vocab:
    f.write(word + "\n")

In [19]:
!python /content/bert_repo/create_pretraining_data.py \
  --input_file=/content/training_data.tsv\
  --output_file=/content/pretraining_data.tsv \
  --vocab_file=/content/vocab.txt \
  --do_lower_case=False \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --masked_lm_prob=0.15 \
  --random_seed=12345 \
  --dupe_factor=5

2020-05-02 20:57:52.113675: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:*** Reading from input files ***
I0502 20:57:54.107752 140465089763200 create_pretraining_data.py:446] *** Reading from input files ***
INFO:tensorflow:  /content/training_data.tsv
I0502 20:57:54.108046 140465089763200 create_pretraining_data.py:448]   /content/training_data.tsv
INFO:tensorflow:*** Writing to output files ***
I0502 21:01:28.049003 140465089763200 create_pretraining_data.py:457] *** Writing to output files ***
INFO:tensorflow:  /content/pretraining_data.tsv
I0502 21:01:28.049268 140465089763200 create_pretraining_data.py:459]   /content/pretraining_data.tsv
INFO:tensorflow:*** Example ***
I0502 21:01:28.050256 140465089763200 create_pretraining_data.py:149] *** Example ***
INFO:tensorflow:tokens: [CLS] Larry Bird one game went 

In [0]:
import json
bert_configs = {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 507971
}

with open('bert_configs.json', 'w', encoding='utf-8') as f:
    json.dump(bert_configs, f, ensure_ascii=False, indent=4)

**Added to run_pretraining.py**

    import subprocess
    import sys
    def install(package):
      subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    install('tensorflow==1.11')
    import tensorflow as tf

**Changed optimization.py:**

    import tensorflow as tf

    class AdamWeightDecayOptimizer(tf.train.Optimizer):

**Changed modeling.py:**

    import tensorflow as tf

In [22]:
!python /content/bert_repo/run_pretraining.py \
  --input_file=/content/pretraining_data.tsv \
  --output_dir=/content/pretraining_output \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=/content/bert_configs.json \
  --train_batch_size=32 \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --num_train_steps=20 \
  --num_warmup_steps=10 \
  --learning_rate=1e-2 \
  --num_train_steps=2 \
  --iterations_per_loop=1

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [0]:
# creating a test case to check for embeddings
!echo 'RIP Kobe Bryant, twitter ' > /tmp/input.txt

In [31]:
# get embeddings from above test case
!python /content/bert_repo/extract_features.py \
  --input_file=/tmp/input.txt\
  --output_file=/content/output.jsonl \
  --vocab_file=/content/vocab.txt \
  --bert_config_file=/content/bert_configs.json \
  --init_checkpoint=/content/pretraining_output/model.ckpt-0 \
  --layers=-1 \
  --max_seq_length=128 \
  --batch_size=8

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [32]:
embedding = []
with open('/content/output.jsonl') as f:
    for line in f:
        embedding.append(json.loads(line))

features = embedding[0]["features"]

for feature in features:
  token_text = feature["token"]
  token_embedding = feature["layers"][0]["values"]
  
  print(f"token: {token_text}")
  print(f"embedding: {token_embedding[:10]}")
  print("\n")

token: [CLS]
embedding: [0.876927, 1.570331, 0.969333, -0.153222, 1.508899, 0.471862, 0.172609, -0.072084, -2.345261, -0.036002]


token: rip
embedding: [1.196484, 0.813006, -1.011249, 0.109201, 0.614897, 0.799736, -0.338902, -0.49217, -2.852371, 1.185313]


token: kobe
embedding: [-0.844613, 1.464169, -0.044304, -0.077628, -0.029806, -0.267319, -0.831179, -0.552, -1.159648, 1.059415]


token: bryant
embedding: [0.286883, 3.198562, -0.358309, -0.435671, 1.075413, -0.891496, -0.652556, -0.86316, -1.87268, 1.076894]


token: ,
embedding: [1.690166, 2.521065, 0.109304, -0.388199, 1.072821, -2.058176, 0.309789, -0.005288, -2.066897, -0.928757]


token: twitter
embedding: [0.431081, 0.542514, 1.014378, -0.164769, 0.978991, -0.76435, 1.215352, -1.853223, -1.824308, -0.322376]


token: [SEP]
embedding: [0.925645, 0.369491, -0.447114, 0.405602, 1.046431, 0.100727, -0.158141, -0.147841, -2.069522, -0.038455]




In [36]:
import shutil
#shutil.move("/content/pretraining_output/model.ckpt-0.data-00000-of-00001", "/content/files/")
#shutil.move("/content/bert_configs.json", "/content/files/")
#shutil.move("/content/vocab.txt", "/content/files/")

!zip -r /content/files.zip /content/files

from google.colab import files
files.download("/content/files.zip")

updating: content/files/ (stored 0%)
updating: content/files/vocab.txt (deflated 48%)
updating: content/files/model.ckpt-0.data-00000-of-00001 (deflated 69%)
updating: content/files/bert_configs.json (deflated 50%)


MessageError: ignored